# Assignment 4 – Promotion Response Prediction (Fast Submission Notebook)

This notebook is optimized for **runtime** in Google Colab.

Speed decisions:
- **No GridSearchCV** in the submission run.
- Uses a **light but strong** feature set.
- Replaces expensive `date.nunique()` frequency with **`txn_cnt`**.
- Limits `nunique` features on transactions to **category** and **brand**.

**Output:** `predict.csv` with two columns: `id`, `active`.


In [ ]:
# =========================
# 0) Install + Imports
# =========================
!pip -q install pyarrow fastparquet

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


## 1) Mount Google Drive and set paths

Place these files in the same folder:

- `transactions.parquet`
- `promos.parquet`
- `train_history.parquet`
- `test_history.parquet`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# CHANGE THIS to your folder
DATA_DIR = "/content/drive/MyDrive/RSM8421_Assignment4"  # <- edit me

transactions_path = os.path.join(DATA_DIR, "transactions.parquet")
promos_path = os.path.join(DATA_DIR, "promos.parquet")
train_history_path = os.path.join(DATA_DIR, "train_history.parquet")
test_history_path = os.path.join(DATA_DIR, "test_history.parquet")


## 2) Load data

In [ ]:
transactions = pd.read_parquet(transactions_path)
promos = pd.read_parquet(promos_path)
train_history = pd.read_parquet(train_history_path)
test_history = pd.read_parquet(test_history_path)

print("transactions:", transactions.shape)
print("promos:", promos.shape)
print("train_history:", train_history.shape)
print("test_history:", test_history.shape)


## 3) Column detection helpers

In [ ]:
def find_first_existing(df, candidates, name="column"):
    for c in candidates:
        if c in df.columns:
            return c
    raise ValueError(
        f"Cannot find {name}. Tried: {candidates}. "
        f"Available cols: {list(df.columns)[:50]} ..."
    )

# customer id column
ID_COL = find_first_existing(train_history, ["id", "customer_id", "cust_id"], "customer id")

# promo key column in history tables (sample notebooks often use 'promo')
PROMO_COL = find_first_existing(train_history, ["promo", "offer", "promotion", "offer_id"], "promo key")

# label column
Y_COL = find_first_existing(train_history, ["active", "response", "responded", "label"], "label")

print("Detected ID_COL:", ID_COL)
print("Detected PROMO_COL:", PROMO_COL)
print("Detected Y_COL:", Y_COL)


## 4) Fast customer features from transactions

In [ ]:
DATE_COL = find_first_existing(transactions, ["date", "transaction_date", "dt"], "transaction date")
AMT_COL  = find_first_existing(transactions, ["amt", "amount", "spend", "sales"], "transaction amount")
QTY_COL  = find_first_existing(transactions, ["qty", "quantity", "units"], "transaction quantity")

# Optional categorical columns (may not exist)
CAT_COL = None
BRAND_COL = None

for c in ["category", "cat", "category_id"]:
    if c in transactions.columns:
        CAT_COL = c
        break

for c in ["brand", "brand_id"]:
    if c in transactions.columns:
        BRAND_COL = c
        break

# Keep only needed columns to speed up groupby
use_cols = [ID_COL, DATE_COL, AMT_COL, QTY_COL]
for opt in [CAT_COL, BRAND_COL]:
    if opt is not None:
        use_cols.append(opt)

transactions = transactions[use_cols].copy()
transactions[DATE_COL] = pd.to_datetime(transactions[DATE_COL], errors="coerce")

max_date = transactions[DATE_COL].max()
transactions["last_purchase_days"] = (max_date - transactions[DATE_COL]).dt.days

grp = transactions.groupby(ID_COL, sort=False)

agg_dict = {
    "recency": ("last_purchase_days", "min"),
    "monetary": (AMT_COL, "sum"),
    "avg_amt": (AMT_COL, "mean"),
    "total_qty": (QTY_COL, "sum"),
    "txn_cnt": (AMT_COL, "size"),  # cheap frequency proxy
}

if CAT_COL is not None:
    agg_dict["unique_categories"] = (CAT_COL, "nunique")

if BRAND_COL is not None:
    agg_dict["unique_brands"] = (BRAND_COL, "nunique")

trans_features = grp.agg(**agg_dict).reset_index()

# Fill missing optional columns if they didn't exist
if "unique_categories" not in trans_features.columns:
    trans_features["unique_categories"] = 0
if "unique_brands" not in trans_features.columns:
    trans_features["unique_brands"] = 0

print("trans_features:", trans_features.shape)
trans_features.head()


## 5) Merge customer + promo features

In [ ]:
features = train_history.merge(trans_features, on=ID_COL, how="left")
predict_features = test_history.merge(trans_features, on=ID_COL, how="left")

# promo key in promos table
PROMO_KEY_PROMOS = find_first_existing(
    promos, [PROMO_COL, "promo", "offer", "promotion", "offer_id"], "promo key in promos"
)

features = features.merge(promos, left_on=PROMO_COL, right_on=PROMO_KEY_PROMOS, how="left")
predict_features = predict_features.merge(promos, left_on=PROMO_COL, right_on=PROMO_KEY_PROMOS, how="left")

print("features:", features.shape)
print("predict_features:", predict_features.shape)


## 6) Select promo columns + impute missing values

In [ ]:
promo_num_candidates = ["promoqty", "promoval"]
promo_cat_candidates = ["category", "brand", "manufacturer"]

promo_num_cols = [c for c in promo_num_candidates if c in features.columns]
promo_cat_cols = [c for c in promo_cat_candidates if c in features.columns]

print("Promo numeric cols:", promo_num_cols)
print("Promo categorical cols:", promo_cat_cols)

def impute_basic(df):
    if "recency" in df.columns:
        df["recency"] = df["recency"].fillna(365)

    for c in ["monetary", "avg_amt", "total_qty", "txn_cnt", "unique_categories", "unique_brands"]:
        if c in df.columns:
            df[c] = df[c].fillna(0)

    for c in promo_num_cols:
        df[c] = df[c].fillna(0)

    return df

features = impute_basic(features)
predict_features = impute_basic(predict_features)


## 7) Consistent encoding for categorical promo fields

In [ ]:
def encode_categoricals(train_df, pred_df, cols):
    if not cols:
        return train_df, pred_df

    combo = pd.concat([train_df[cols], pred_df[cols]], axis=0)

    for c in cols:
        combo[c] = combo[c].astype("category")
        cats = combo[c].cat.categories

        train_df[c] = pd.Categorical(train_df[c], categories=cats).codes
        pred_df[c]  = pd.Categorical(pred_df[c], categories=cats).codes

        # -1 indicates missing/unseen categories
        train_df[c] = train_df[c].replace(-1, -1)
        pred_df[c]  = pred_df[c].replace(-1, -1)

    return train_df, pred_df

features, predict_features = encode_categoricals(features, predict_features, promo_cat_cols)


## 8) Final feature list

In [ ]:
base_cols = [
    "recency", "monetary", "txn_cnt",
    "avg_amt", "total_qty",
    "unique_categories", "unique_brands"
]

x_cols = [c for c in base_cols if c in features.columns] + promo_num_cols + promo_cat_cols

print("Final x_cols:", x_cols)


## 9) Train/validation and fast fixed model

In [ ]:
X = features[x_cols].copy()
y = features[Y_COL].copy()

strat = y if y.nunique() == 2 else None

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=strat
)

print("Train size:", X_train.shape, "Val size:", X_val.shape)
print("Class balance (train):", y_train.value_counts(normalize=True).round(3).to_dict())

best_rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_leaf=4,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
)

best_rf_model.fit(X_train, y_train)

val_pred = best_rf_model.predict_proba(X_val)[:, 1]
val_auc = roc_auc_score(y_val, val_pred)
print("Validation ROC AUC:", round(val_auc, 4))


## 10) Fit on all labeled data and generate submission

In [ ]:
best_rf_model.fit(X, y)

X_predict = predict_features[x_cols].copy()
pred_proba = best_rf_model.predict_proba(X_predict)[:, 1]

predict_out = predict_features[[ID_COL]].copy()
predict_out.columns = ["id"]  # enforce required output name
predict_out["active"] = pred_proba

print(predict_out.head())
print("Prediction rows:", len(predict_out), "Expected:", len(test_history))

out_path = os.path.join(DATA_DIR, "predict.csv")
predict_out.to_csv(out_path, index=False)
print("Saved:", out_path)
